In [1]:
# !git clone https://github.com/Lilcob/dataset_for_lg.git

In [2]:
# !unzip './dataset_for_lg/dataset_lg.zip'

In [3]:
dataset_path = './dataset'

## 데이터 전처리

In [4]:
import os
import glob
#pillow 와 opencv가 이미지 라이브러리 처리에 양대산맥
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split

dataset = []
labels = []

folder_name = ['lung','lung_cancer']
for idx,folder in enumerate(folder_name):
    img_dir_name = os.path.join(dataset_path,folder)
    files = glob.glob(img_dir_name + '/*')
    # print(files)
    for file in files:
        img = Image.open(file)
        # print(img) # 모두 이미지의 파일의 크기가 다를 것 
        # 흑백이면 depth가 1, RGB는 3이기때문에 모두 3으로 맞추기
        img = img.convert('RGB')
        #사이즈 동일하게 변경
        img = img.resize((64,64))
        data = np.asarray(img)
        dataset.append(data)
        #라벨 저장, idx lung 폴더는 0, lung_cancer 1
        labels.append(idx)


print(type(data))
print(data.size)
# print(dataset[0], labels[0])

#리스트형태를 Numpy 형태로 바꿀것
datasets = np.array(dataset)
labels = np.array(labels)

<class 'numpy.ndarray'>
12288


In [5]:
x_train, x_test, y_train, y_test = train_test_split(datasets,labels,test_size=0.1)
print(x_train.shape) # 207개의 데이터, 64x64 크기, depth 는 3 
print(y_train.shape) # 23개의 데이터 

(207, 64, 64, 3)
(207,)


In [ ]:
#함수형 방식 , functional 방식
x_train = x_train / 255.0
x_test = x_test / 255.0
import tensorflow as tf
from tensorflow.keras.layers import Input
input_img = Input(shape = (64,64,3) ,name='input_start')

from tensorflow.keras.layers import Conv2D, MaxPooling2D
tower_0 = Conv2D(64, (1,1) , padding='same',activation='relu',name = 'tower_0_start')(input_img)
####
tower_1 = Conv2D(64, (1,1) , padding='same',activation='relu', name = 'tower_1_start')(input_img)
tower_1 = Conv2D(64, (3,3) , padding='same',activation='relu')(tower_1)
###

tower_2 = MaxPooling2D((3,3) , strides=(1,1), padding='same')(input_img)
tower_2 = Conv2D(64, (3,3) , padding='same',activation='relu')(tower_2)


tower_3 = Conv2D(64, (1,1) , padding='same',activation='relu')(input_img)
tower_3 = Conv2D(64, (3,3) , padding='same',activation='relu')(tower_3)
tower_3 = Conv2D(64, (3,3) , padding='same',activation='relu')(tower_3)
##
output = tf.keras.layers.concatenate([tower_0,tower_1,tower_2,
                                      tower_3], axis = 3)

from tensorflow.keras.layers import Flatten, Dense
output = Flatten()(output)
out = Dense(1, activation = 'sigmoid')(output)
#out = Dense(2, activation = 'softmax')(output)

from tensorflow.keras.models import Model
model = Model(inputs = input_img, outputs = out)

print(model.summary())
from tensorflow.keras.optimizers import SGD
momentum_param = 0.9
lrate = 0.1
sgd = SGD(learning_rate = lrate, momentum = momentum_param)

model.compile(loss= 'binary_crossentropy', optimizer = 'adam')
model.fit(x_train, y_train, 
          validation_data= (x_test,y_test),
          epochs=10, batch_size=64)